In [1]:
import gc
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from typing import Iterable
import random
import warnings
from transferDataset import TransferDataset
from sklearn.metrics import r2_score
import torchmetrics
import torch
import numpy as np
from tqdm import tqdm
import os

In [2]:
path = './compare'
# path = './compare'
listdir = os.listdir(path)

listdir.sort()
join = os.path.join(path, listdir[0]).split('.csv')[0]
join

'./compare/Eawag_XBridgeC18'

In [3]:
dataset = TransferDataset(join)
print(join)
dataset_name = join.split('/')[2]

./compare/Eawag_XBridgeC18


In [5]:
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


kfold = 10
seed = 1234

fold_size = len(dataset) // kfold
fold_reminder = len(dataset) % kfold
split_list = [fold_size] * kfold
for reminder in range(fold_reminder):
    split_list[reminder] = split_list[reminder] + 1
set_seed(seed)
split = random_split(dataset, split_list)
for fold in range(kfold):
    best_train_mae = 99999
    best_test_mae = 99999
    set_seed(seed)
fold = 0

In [6]:
model = torch.load(f'./models/{dataset_name}/fold{fold}.pkl')

In [8]:
test_dataset = split[fold]
train_list = []
for m in range(kfold):
    if m != fold:
        train_list.append(split[m])
train_dataset = torch.utils.data.ConcatDataset(train_list)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True,
                         num_workers=2, pin_memory=True,
                         prefetch_factor=2, drop_last=True, persistent_workers=True, )
test_loader = list(test_loader)

In [9]:
test_loader[0]

DataBatch(x=[28, 34], edge_index=[2, 56], edge_attr=[56, 5], y=[1], fingerprint=[2048], inchi=[1], formula=[1], batch=[28], ptr=[2])

In [10]:
data = test_loader[0]
data.inchi

['InChI=1S/C8H14ClN5/c1-4-10-7-12-6(9)13-8(14-7)11-5(2)3/h5H,4H2,1-3H3,(H2,10,11,12,13,14)']

In [13]:
formula = data.formula[0]

In [14]:
f'search_5_datasets/{formula}.csv'

'search_5_datasets/C8H14ClN5.csv'

In [17]:
next_data_path = f'search_5_datasets/{formula}.csv'
os.path.exists(f'search_5_datasets/{formula}.csv')

True

In [20]:
from transferDataset import PredictionDataset

predict_dataset = PredictionDataset(f'search_5_datasets/{formula}')

Processing...
/home/jun/projects/multiGPU/transferDataset.py:219: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  hybridization_encoder.fit(torch.range(0, len(hybridization_list) - 1).unsqueeze(-1))
/home/jun/projects/multiGPU/transferDataset.py:223: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  atom_encoder.fit(torch.range(0, len(atom_list) - 1).unsqueeze(-1))
/home/jun/projects/multiGPU/transferDataset.py:226: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  chirarity_encoder.fit(torch.range(0, le

In [21]:
model.eval()

MyNet(
  (in_linear): Linear(in_features=34, out_features=512, bias=True)
  (conv1): GraphTransformerBlock(
    (conv): GATConv(512, 512, heads=3)
    (linear): Linear(in_features=1536, out_features=512, bias=True)
    (layerNorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (conv2): GraphTransformerBlock(
    (conv): GATConv(512, 512, heads=3)
    (linear): Linear(in_features=1536, out_features=512, bias=True)
    (layerNorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (conv3): GraphTransformerBlock(
    (conv): GATConv(512, 512, heads=3)
    (linear): Linear(in_features=1536, out_features=512, bias=True)
    (layerNorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (conv4): GraphTransformerBlock(
    (conv): GATConv(512, 512, heads=3)
    (linear): Linear(in_features=1536, out_features=512, bias=True)
    (layerNorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (conv5): GraphTransformerBlock(
    (conv): GATConv(512, 512

In [30]:
predict_loader = DataLoader(predict_dataset, batch_size=1024, shuffle=True,
                            num_workers=2, pin_memory=True,
                            prefetch_factor=2, persistent_workers=True, )

In [31]:
for pred_data in predict_loader:
    pass

In [32]:
pred_data

DataBatch(x=[4928, 34], edge_index=[2, 9946], edge_attr=[9946, 5], fingerprint=[360448], inchi=[176], batch=[4928], ptr=[177])

In [33]:
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pred_data.to(device)
pred = model(pred_data)
pred

tensor([335.3817, 306.4652, 216.0673, 238.3221, 518.3769, 204.0869, 536.5920,
        127.9471, 555.7635, 474.3678, 495.4148, 156.3196, 294.4536, 239.6917,
        226.2868, 556.6192, 139.2700, 390.4460, 290.1817, 484.7779, 450.2365,
        300.8079, 454.6169, 531.6185, 510.3573, 326.6182, 221.3356, 115.5481,
        227.7112, 390.0638, 387.7846, 441.9623, 556.9523, 131.7753, 394.9728,
        378.1610, 508.7184, 218.6313,  77.9734, 383.0794, 400.7393, 516.5687,
        136.4600, 441.3495, 488.0632, 442.9339,  76.6999, 241.1502, 259.8111,
        312.1990, 385.0972, 240.5607, 297.4230, 133.1615, 294.1974, 381.9258,
         75.0953, 517.6558, 199.3933, 552.7399, 510.9384, 489.2650, 513.9898,
        260.5828, 446.1741, 182.8530, 109.2841, 496.6371, 429.3144, 133.8186,
        533.1366, 604.9606, 506.5281, 492.3094, 535.4645, 517.7291, 316.4485,
        223.0589, 216.0197, 463.3930,  75.7677, 242.7956, 340.2125, 104.8407,
        396.6420,  72.9419, 183.2796, 149.3518, 358.1777,  78.03

In [35]:
pred.reshape(-1)

torch.Size([176])

In [36]:
range(len(pred.reshape(-1)))

range(0, 176)

In [47]:
import pandas as pd
from tqdm import tqdm

for fold in range(kfold):
    best_train_mae = 99999
    best_test_mae = 99999
    set_seed(seed)
    test_dataset = split[fold]
    train_list = []
    for m in range(kfold):
        if m != fold:
            train_list.append(split[m])
    train_dataset = torch.utils.data.ConcatDataset(train_list)

    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True,
                             num_workers=2, pin_memory=True,
                             prefetch_factor=2, drop_last=True, persistent_workers=True, )
    test_loader = list(test_loader)
    frame = pd.DataFrame(columns=['inchi_true', 'rt_true', 'inchi', 'rt_pred'])
    frame_index = 0
    for data in tqdm(test_loader):
        formula = data.formula[0]
        next_data_path = f'./search_5_datasets/{formula}'
        predict_dataset = PredictionDataset(next_data_path)
        predict_loader = DataLoader(predict_dataset, batch_size=1024, shuffle=True,
                                    num_workers=2, pin_memory=True,
                                    prefetch_factor=2, persistent_workers=True)

        inchi_true = data.inchi
        rt_true = data.y.cpu().item()
        for data_for_pred in predict_loader:
            data_for_pred.to(device)
            pred = model(data_for_pred)
            reshape = pred.reshape(-1)
            for i in range(len(reshape)):
                pred_rt = reshape[i]
                inchi = data_for_pred[i].inchi
                frame.loc[frame_index] = {
                    'inchi_true': inchi_true,
                    'rt_true': rt_true,
                    'inchi': inchi,
                    'rt_pred': pred_rt.cpu().item()
                }
                frame_index = frame_index + 1

frame.to_csv(f'results/{dataset_name}_results.csv')

 76%|████████████████████████████████████████████████████████████████████████████████████████████████████▋                                | 28/37 [05:07<01:38, 10.96s/it]


KeyboardInterrupt: 